# Set Ups

In [1]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
from pkg_resources import ContextualVersionConflict

# from rich.console import Console

import requests
import json

import time
from sqlalchemy import delete
from tqdm import tqdm
import re
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
# connet to graphql
sample_transport=RequestsHTTPTransport(
    url='http://localhost:4001/graphql',
    use_json=True,
    headers={
        "Content-type": "application/json",
    },
    verify=False
)

client = Client(
    #execute_timeout=20,
    transport=sample_transport,
    fetch_schema_from_transport=True,
)

session = requests.Session()
session.auth = ('admin', "secret")

#hostname = 'http://ec2-3-64-237-95.eu-central-1.compute.amazonaws.com:8080/tilt/tilt'
#auth = session.post(hostname)
#response = json.loads(session.get(hostname).content)

#tilt = json.loads(requests.get('http://ec2-3-64-237-95.eu-central-1.compute.amazonaws.com:8080/tilt/tilt').content)
#print("Fetch Successful!", response)


json_file = open('/Users/johannes/Desktop/TU/playground/tilt')
tilt = json.load(json_file)

In [3]:
hostname = 'http://ec2-3-64-237-95.eu-central-1.compute.amazonaws.com:8080/tilt/tilt'
auth = session.post(hostname)
response = session.get(hostname).content
json_tilt = json.loads(response)

# Load Data from TILT HUB into NEO4J

In [4]:

def create_node(parent):

    """
    func:   create_node
            Recursively goes through JSON and builds a mutation to generate the neo4j graph
            Base case is when an entry is a string
    """

    def rem_dq(uncleaned):

        """
        func:   rem_dq
                removes double quotes. 
                Needed to avoid errors in upload
        """
        if isinstance(uncleaned,str):
            return re.sub('"', '', uncleaned)
        else:
            return uncleaned

    #mutation string
    label_string = ""

    #iterate through elements of dictionary or list
    for i, key in enumerate(parent):

        #if key == 'rightToRectificationOrDeletion':
            #label_string += " HERE "
            
        try:
            if key[0] == "$": # the dollar sign infront of the $oid is not recognised so we have to remove it
                label_string += f'{key[1:]}: "{rem_dq(parent[key])}"'


            # if key is a list
            elif isinstance(parent[key], list):

            
                #if list item is empty
                if len(parent[key]) == 0:
                    
                    #label_string += f'{key}: "{parent[key]}"'
                    label_string += "" # just add nothing. not ideal as we lose out on recording missing information but easier to fix than trying to fix missfilled fields in tilt

                #two types of entries: nodes or strings
                else:

                    # When we get lists with multiple same node entries, we make a list of nodes. 
                    # There we can only need one create statement and then close it afterwards
                    label_string += key+ ':{create:['
                    node_list = True

                    for list_item in parent[key]:
                        
                        # some entries are a string wrapped in a lists instead of a node.
                        if type(list_item) == type(""): 
                            #deleted = label_string[-len(f'{key}:{{create:['):]
                            label_string = label_string[:-len(f'{key}:{{create:[')]                             
                            label_string += f'{key}: "{rem_dq(list_item)}"'
                            node_list = False

                        # if the list items are dictionaries make a new node
                        else: 
                            label_string += f'{{{create_node(list_item)}}},'

                    #
                    if node_list:
                        label_string = label_string[:-1]+']}'
            
                
            # if the element is a dictionary
            elif isinstance(parent[key], dict): 
                
                #create the next node for this dict
                label_string += f'{key}:{{create:[{{{create_node(parent[key])}}}]}}'

            else: # base case
                
                if i < len(parent)-1:

                    label_string += f'{key}: "{rem_dq(parent[key])}", '
                    
                else:
                    label_string += f'{key}: "{rem_dq(parent[key])}"'
                
        except Exception as ex:
            label_string += "BIG ERROR HERE"  
            print(ex)  

    #certain characters (like tilda) are rejected by gql
    label_string = re.sub('~', "", label_string)
    label_string = label_string.strip().replace('\n', "")
    
    return label_string

    

In [5]:
print("mutation { createtilts( input: [ { " + create_node(json_tilt[5]) + "}] ) {tilts {meta { name } } }}")

mutation { createtilts( input: [ { _id:{create:[{oid: "62e14a7ce7dfaf542d39af87"}]}meta:{create:[{_id: "90eeb36e-3d54-4b0a-88b7-f4565cd66920", name: "Zoom", created: "2021-07-17T10:17:13.671015", modified: "2021-07-17T10:17:13.671037", version: "1", language: "en", status: "active", url: "https://zoom.us/privacy", _hash: "54840059bb2a41fcca06c6381e0e6c0708ae0e0d4e9757612fe9d646820f7c1f"}]}controller:{create:[{name: "Zoom Video Communications, Inc.", country: "US", address: "55 Almaden Blvd, Suite 600San Jose, CA 95113", division: "Data", representative:{create:[{name: "None", email: "None", phone: "None"}]}}]}dataProtectionOfficer:{create:[{email: "privacy@zoom.us", phone: "None", country: "US", name: "None", address: "Zoom Video Communications, Inc.Attention: Data Privacy Officer55 Almaden Blvd, Suite 600San Jose, CA 95113"}]}dataDisclosed:{create:[{_id: "0112176c-e991-444e-b0ab-780f7bceaa43", category: "Account Information", purposes:{create:[{purpose: "Provide Zoom Products and Serv

In [6]:
tilt_mutations = []
for tilt_entry in tqdm(tilt):
    tilt_entry  
    tilt_mutations.append("mutation { createtilts( input: [ { " + create_node(tilt_entry) + "}] ) {tilts {meta { name } } }}")

100%|██████████| 73/73 [00:00<00:00, 1677.51it/s]


In [7]:
successes = 0
for i, query_string in enumerate(tqdm(tilt_mutations)):
    #print("Query String: ", query_string)
    try:
        query = gql(query_string)
        client.execute(query)
        successes += 1
    except Exception as syn:
        print(syn)
        print("Error With Tilt No.", i)

print("Successfully Uploaded:", 100*(successes/(i+1)), "per. of tilts")

100%|██████████| 73/73 [05:15<00:00,  4.32s/it]

Successfully Uploaded: 100.0 per. of tilts


# Analysis 

In [8]:
from neo4j import GraphDatabase

In [9]:
class make_queries:

    """
    class:  exampleTilt
    input:  - uri: bolt uri from neo4j
            - auth: credentials for neo4j
            - no_nodes: number of nodes to create 
            - comp_df:  dataframe with company names and websites
            - mu_poisson_*: average no. of samples for poisson distribution
            - reset: whether to delete all nodes in database (default False, meant for experimentation)

    """

    def __init__(self, uri, auth,  reset = False):
        self.driver = GraphDatabase.driver(uri, auth=auth)
        self.reset = reset


    def close(self):
        self.driver.close()
        
    # Match and display all friendships.
    @classmethod
    def print_friendships(cls, tx):
        result = tx.run("MATCH (a)-[:KNOWS]->(b) RETURN a.name, b.name")
        for record in result:
            print("{} knows {}".format(record["a.name"], record["b.name"]))

    @classmethod
    def delete_nodes(cls, tx):
        tx.run("MATCH (n) detach delete n")

    
    def get_sorensen(self, tx, threshold = .9):
        tx.run("MATCH (m:meta), (r:recipient) "
                f"WHERE apoc.text.sorensenDiceSimilarity(apoc.text.clean(m.name), apoc.text.clean(r.name), 'en') > {threshold} "
                "MERGE (m)-[c:has_sorensen]->(r)")


    def run_query(self, tx, cyper_statement):
        
        result = tx.run(cyper_statement)
        return [point.data() for point in result] 
        

    def main(self):
        saved_bookmarks = []  # To collect the session bookmarks

        if self.reset:
            assurance = input("WARNING DELETING ALL DATA FROM THE GRAPH, TO CONTINUE WRITE 'yes'")
            if assurance == 'yes':
                with self.driver.session() as session_del:
                    session_del.write_transaction(self.delete_nodes)
                    saved_bookmarks.append(session_del.last_bookmark())

        #with self.driver.session() as session_sorensen: 
        #    session_sorensen.write_transaction(self.get_sorensen)

infos = make_queries(uri = 'bolt://localhost:7687', 
                    auth = ('neo4j', 'letmein'), 
                    reset=False)

In [10]:
list_of_categories = [i['d.category'] for i in infos.driver.session().write_transaction(infos.run_query, "match (d:dataDisclosed) return d.category") if i['d.category'] != None ]

with open('categories_tilt', 'wb') as fp:
    pickle.dump(np.unique(np.asarray(list_of_categories)), fp)

In [18]:
len(np.unique(np.asanyarray(list_of_categories)))/73

7.315068493150685

In [11]:
list_of_purposes = [i['p.purpose'] for i in infos.driver.session().write_transaction(infos.run_query, "match (p:purposes) return p.purpose") if i['p.purpose'] != None ]

with open('purpose_tilt', 'wb') as fp:
    pickle.dump(np.unique(np.asarray(list_of_purposes)), fp)

In [19]:
len(np.unique(np.asarray(list_of_purposes)))/73

1.5342465753424657